# **FEATURE IMPORTANCE**


First need to get the database from PostGRES

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/train_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("train_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+---------+---------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+---------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+-----------+------------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-------+------+------+---------+-----------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------

Forest of Trees Feature Selection


In [4]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig

In [5]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
#####################Create the feature data and the target

#Feature DataFrame is X
X = df.drop(columns=["Id", "SalePrice"])


#Target Dataframe is y
y = df["SalePrice"]


#####################Create train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

#Scale
X_train_scaler = StandardScaler().fit(X_train)
X_test_scaler = StandardScaler().fit(X_test)
X_train_scaled = X_train_scaler.transform(X_train)
X_test_scaled = X_test_scaler.transform(X_test)

In [7]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train_scaled, y_train)
print(f' Training score: {rf.score(X_train_scaled, y_train)}')
print(f' Test score: {rf.score(X_test_scaled, y_test)}')

 Training score: 1.0
 Test score: 0.0


In [8]:
importances = rf.feature_importances_
features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)

[(0.03551912568306011, 'PoolArea'),
 (0.03278688524590164, 'TotalBsmtSF'),
 (0.03278688524590164, 'ExterQual'),
 (0.03278688524590164, 'BsmtFinSF1'),
 (0.030054644808743168, 'Neighborhood_NoRidge'),
 (0.030054644808743168, 'LotFrontage'),
 (0.030054644808743168, 'BsmtFinType2'),
 (0.0273224043715847, 'WoodDeckSF'),
 (0.0273224043715847, 'GarageType_Attchd'),
 (0.02459016393442623, 'PoolQC'),
 (0.02459016393442623, 'LotArea'),
 (0.02459016393442623, 'Fireplaces'),
 (0.02459016393442623, 'Exterior2nd_Stucco'),
 (0.02459016393442623, 'Condition1_Norm'),
 (0.02185792349726776, 'Exterior1st_Wd Sdng'),
 (0.02185792349726776, 'Exterior1st_HdBoard'),
 (0.02185792349726776, 'BsmtHalfBath'),
 (0.01912568306010929, 'LowQualFinSF'),
 (0.01912568306010929, 'GarageFinish'),
 (0.01912568306010929, 'Exterior2nd_ImStucc'),
 (0.01912568306010929, 'Exterior1st_Stucco'),
 (0.01912568306010929, 'BsmtQual'),
 (0.01912568306010929, 'BsmtFinSF2'),
 (0.01912568306010929, 'BsmtExposure'),
 (0.01912568306010929,

ValueError: ignored